**Import Necessary Libraries**

In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

**Define Functions for model and data preperation.**

In [32]:
def create_lstm_model(input_shape):
    model=Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=25),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss=lambda y_true, y_pred: weighted_loss(y_true, y_pred))
    return model

In [33]:
def weighted_loss(y_true, y_pred):
    weights = tf.range(1, tf.shape(y_true)[0]+1, dtype=tf.float32)
    #Exponential Weights
    weights = tf.math.exp(weights/ tf.reduce_max(weights))
    loss = tf.keras.losses.MeanSquaredError()(y_true, y_pred)
    return tf.reduce_mean(loss * weights) 

In [34]:
def prepare_data(df, sequence_length=60):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))

    X, y = [],[]
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, scaler

**Load and Prepare Data**

In [35]:
def load_stock_data(filepath):
    df = pd.read_csv(filepath)
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df.set_index('Date', inplace=True)
    return df

df = load_stock_data("data/nvidia_stock.csv")
X, y , scaler = prepare_data(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = False)

**Train the LSTM Model**

In [36]:
model = create_lstm_model((X_train.shape[1], 1))
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50


c:\Users\athar\anaconda3\envs\data_science\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


146/146 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 6.7582e-05 - val_loss: 9.4065e-06
Epoch 2/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 1.3054e-06 - val_loss: 8.6428e-06
Epoch 3/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 1.3367e-06 - val_loss: 8.9700e-06
Epoch 4/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 1.4674e-06 - val_loss: 7.7274e-06
Epoch 5/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 1.0433e-06 - val_loss: 2.3997e-05
Epoch 6/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 1.1222e-06 - val_loss: 3.4526e-05
Epoch 7/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 1.2238e-06 - val_loss: 6.7115e-06
Epoch 8/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 2.3846e-06 - val_loss: 1.0381e-05
Epoch 9/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 1.0781e-06 - val_loss: 8.6564e-06
Epoch 10/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 1.0073e-06 - val_loss: 6.6755e-06
Epoch 11/50
146/146 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - los